<a href="https://colab.research.google.com/github/hjc2/pa2_regretmin/blob/master/HJC_PA2_Regret_Min.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PA2 — Regret Minimization in Repeated Normal-Form Games
*Intelligent Agents — Programming Assignment 2*

**Tasks**
1. Compute **average strategy** and **external regret** for the row player from a history of play.
2. Implement **Follow-the-Leader (FTL)** against an input column action sequence, and a **self-play** version (both players run FTL).
3. Implement **Multiplicative Weights Update (MWU)** against an input column action sequence, and a **self-play** version.
4. Plot the **distance to a given NE** for the two **self-play** versions on a specified game.

**What to submit:** your completed `.ipynb` on Canvas. Before submitting, do **Runtime → Restart and run all**.

**Conventions**
- Games are two-player normal-form with row payoff matrix `A` (shape `(n_row, n_col)`) and column payoff matrix `B` (same shape).
- Action indices are **0-based**.
- Use only `numpy` and `matplotlib` (no seaborn).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def show(arr, name=None):
    if name:
        print(name, "=", end=" ")
    print(np.array(arr))

### 🧩 Task 1 — Average Strategy & External Regret (row player)

Implement:

- `average_strategy_row(row_actions, n_actions)` → empirical distribution over the row player's actions.
  - Input: list/array of row action indices in `{0,…,n_actions-1}`
  - Output: 1D `np.ndarray` of length `n_actions` summing to 1
  - Edge case: if empty, return the **uniform** distribution.

- `external_regret_row(row_actions, col_actions, A)` → nonnegative float:
$$
R_T \,=\, \max_{a}\sum_{t=1}^T A[a, j_t] - \sum_{t=1}^T A[i_t, j_t].
$$
- Edge case: if empty, return `0.0`.

In [ ]:
def average_strategy_row(row_actions, n_actions):
    """Return the empirical distribution over row actions.
    If no actions, return uniform.
    """
    # TODO: implement
    pass


def external_regret_row(row_actions, col_actions, A):
    """External regret of the row player vs the best fixed row action.
    Regret(T) = max_a sum_t A[a, j_t] - sum_t A[i_t, j_t].
    """
    # TODO: implement
    pass

In [ ]:
# === Public tests: Task 1 ===
def _ok(name, fn):
    try:
        fn(); print(f"✅ {name} — passed")
    except AssertionError as e:
        print(f"❌ {name} — {e}")

def test_avg_empty_uniform():
    n=3
    avg = average_strategy_row([], n)
    assert avg.shape==(n,)
    assert np.allclose(avg.sum(),1.0)
    assert np.allclose(avg, np.ones(n)/n)

def test_avg_counts():
    avg = average_strategy_row([0,2,2,1,2], 3)
    assert np.allclose(avg, [1/5,1/5,3/5])

def test_regret_matching_pennies():
    A = np.array([[+1,-1],[-1,+1]], float)
    row = [0,0,0,0]
    col = [1,1,0,1]
    reg = external_regret_row(row, col, A)
    assert np.isclose(reg, 4.0)

_ok("T1: avg empty uniform", test_avg_empty_uniform)
_ok("T1: avg simple counts", test_avg_counts)
_ok("T1: regret (MP)", test_regret_matching_pennies)

### 🧩 Task 2 — Follow-the-Leader (FTL): opponent-given and self-play

- `expected_payoff_row_against_mix(A, q)` → expected payoffs for each row action vs a column mixed strategy `q`.
- `ftl_row(col_actions, A, tie_break='first', seed=None)` → row action sequence of length `T` (no peeking at `col_actions[t]` when choosing round `t`).
- `ftl_self_play(A, B, T, tie_break='first', seed=None)` → simulate both players running FTL **simultaneously** for `T` rounds.
  - Returns `(row_actions, col_actions)`.

In [ ]:
def expected_payoff_row_against_mix(A, q):
    """Return v[a] = sum_j A[a,j] * q[j] for each row action a."""
    # TODO: implement
    pass


def ftl_row(col_actions, A, tie_break='first', seed=None):
    """FTL for the row player against realized column actions (no peeking)."""
    # TODO: implement
    pass


def ftl_self_play(A, B, T, tie_break='first', seed=None):
    """Self-play: both players run FTL based on the opponent's prefix empirical distribution.
    Return (row_actions, col_actions)."""
    # TODO: implement
    pass

In [ ]:
# === Public tests: Task 2 ===
def _ok2(name, fn):
    try:
        fn(); print(f"✅ {name} — passed")
    except AssertionError as e:
        print(f"❌ {name} — {e}")

def test_expected_row_mix():
    A = np.array([[1,0],[0,1]], float)
    q = np.array([0.25,0.75])
    v = expected_payoff_row_against_mix(A, q)
    assert np.allclose(v, [0.25,0.75])

def test_ftl_row_basic():
    A = np.array([[1,0],[0,1]], float)
    col = [1,1,1,1,1]
    row = ftl_row(col, A, tie_break='first')
    assert row[0] == 0 and np.all(row[1:] == 1)

def test_ftl_self_play_shapes():
    A = np.array([[1,0],[0,1]], float); B = A.copy()
    T = 10
    r,c = ftl_self_play(A,B,T,tie_break='first',seed=42)
    assert r.shape==(T,) and c.shape==(T,)

_ok2("T2: expected payoff vs mix", test_expected_row_mix)
_ok2("T2: ftl_row basic", test_ftl_row_basic)
_ok2("T2: self-play shapes", test_ftl_self_play_shapes)

### 🧩 Task 3 — Multiplicative Weights Update (MWU): opponent-given and self-play

- `mwu_row(col_actions, A, eta, seed=None)` → `(row_actions, probs_over_time)` where `probs_over_time[t] = x_t` (distribution **before** sampling round `t`).
- `mwu_self_play(A, B, T, eta_row, eta_col, seed=None)` → simulate both players using MWU.
  - Returns `(row_actions, col_actions, probs_row_over_time, probs_col_over_time)`.

In [ ]:
def mwu_row(col_actions, A, eta, seed=None):
    """MWU for the row player against realized column actions.
    Return (row_actions, probs_over_time)."""
    # TODO: implement
    pass


def mwu_self_play(A, B, T, eta_row, eta_col, seed=None):
    """Self-play MWU for T rounds.
    Return (row_actions, col_actions, probs_row_over_time, probs_col_over_time)."""
    # TODO: implement
    pass

In [ ]:
# === Public tests: Task 3 ===
def _ok3(name, fn):
    try:
        fn(); print(f"✅ {name} — passed")
    except AssertionError as e:
        print(f"❌ {name} — {e}")

def test_mwu_shapes():
    A = np.array([[1,0],[0,1]], float)
    col = [0,1,0,1]
    row, P = mwu_row(col, A, eta=0.2, seed=631)
    assert row.shape==(len(col),,) and P.shape==(len(col), A.shape[0])
    assert np.allclose(P.sum(axis=1),1.0)

def test_mwu_self_play_shapes():
    A = np.array([[1,0],[0,1]], float); B = A.copy()
    T=12
    r,c,Pr,Pc = mwu_self_play(A,B,T,eta_row=0.2,eta_col=0.2,seed=7)
    assert r.shape==(T,) and c.shape==(T,)
    assert Pr.shape==(T,A.shape[0]) and Pc.shape==(T,B.shape[1])

_ok3("T3: mwu_row shapes", test_mwu_shapes)
_ok3("T3: mwu_self_play shapes", test_mwu_self_play_shapes)

### 🧩 Task 4 — Distance to NE for Self-Play (FTL & MWU)

Given a game `(A,B)` and a target **row NE mix** `x_star`, plot the distance
$ d_t = \|\bar{x}_t - x^*\|_1 $ for the **self-play** FTL and MWU runs.

- `average_strategy_over_time_row(row_actions, n_actions)` → `(T, n_row)` cumulative averages.
- `distance_to_NE(avg_over_time, x_star, ord='l1')` → vector `d` of length `T`.
- `compare_selfplay_convergence(A, B, T, x_star, eta_row, eta_col, seed=None)`
  - Run `ftl_self_play` and `mwu_self_play` on the same game/horizon/seed.
  - Return `{'d_ftl': d_ftl, 'd_mwu': d_mwu}` and **plot** both curves.

In [ ]:
def average_strategy_over_time_row(row_actions, n_actions):
    """Return cumulative average strategy over time as an array of shape (T, n_actions)."""
    # TODO: implement
    pass


def distance_to_NE(avg_over_time, x_star, ord='l1'):
    """Return d_t = || avg_over_time[t] - x_star ||_ord over t."""
    # TODO: implement
    pass


def compare_selfplay_convergence(A, B, T, x_star, eta_row, eta_col, seed=None):
    """Run FTL and MWU self-play and compare distance-to-NE for the row player.
    Return {'d_ftl': d_ftl, 'd_mwu': d_mwu}. Also plot both curves."""
    # TODO: implement
    pass

In [ ]:
# === Public tests: Task 4 ===
def _ok4(name, fn):
    try:
        fn(); print(f"✅ {name} — passed")
    except AssertionError as e:
        print(f"❌ {name} — {e}")

def test_avg_over_time_tiny():
    avg = average_strategy_over_time_row([0,2,2,1,2], 3)
    expected = np.array([
        [1,0,0],
        [0.5,0,0.5],
        [1/3,0,2/3],
        [0.25,0.25,0.5],
        [0.2,0.2,0.6],
    ])
    assert np.allclose(avg, expected)

def test_distance_basic():
    avg = np.array([[1,0],[0.6,0.4],[0.5,0.5]], float)
    x_star = np.array([0.5,0.5], float)
    d = distance_to_NE(avg, x_star, ord='l1')
    assert d.shape==(3,)
    assert np.isclose(d[-1], 0.0)

_ok4("T4: avg over time tiny", test_avg_over_time_tiny)
_ok4("T4: distance basic", test_distance_basic)

def test_matching_pennies_convergence():
    # Row payoff matrix (col gets the negative)
    A = np.array([[+1, -1],
                  [-1, +1]], float)
    B = -A
    x_star = np.array([0.5, 0.5])
    T = 400

    out = compare_selfplay_convergence(A, B, T, x_star,
                                       eta_row=0.1, eta_col=0.1,
                                       seed=631)
    d_ftl, d_mwu = out["d_ftl"], out["d_mwu"]

    # Sanity: correct shapes
    assert d_ftl.shape == (T,)
    assert d_mwu.shape == (T,)

    # FTL tends to cycle away from NE; MWU should average closer in tail
    tail_ftl = np.mean(d_ftl[-100:])
    tail_mwu = np.mean(d_mwu[-100:])
    print("Avg tail distances:", tail_ftl, tail_mwu)

    assert tail_mwu < tail_ftl + 0.2, "MWU should not be worse than FTL in MP"
    assert tail_mwu < 0.5, "MWU should be reasonably close to NE on average"

test_matching_pennies_convergence()


✏️ Methods Note (Reflection)

Please add a short note (5–10 sentences) reflecting on your work for this assignment. You might address questions such as:

- Which parts of the code did you find most challenging to implement, and why?

- How did you verify that your implementations were correct? (e.g., reasoning, manual checks, extra experiments)

- Did you notice any interesting behavior of the algorithms when you tested them?

- If you had more time, how might you improve or extend your code?

- What did you learn about regret minimization or equilibria through coding these tasks?

This note will not be graded for correctness, but for thoughtful engagement.